**Capstone Project -- The Battle of Neighborhoods**


**Introduction : Bussiness Problem**

In this project we will try to find the optimal location for a resturant . Specifically , this will be helpful for stakeholder who are going to open new Italian Resturant in the Kormangla area of Banglore, India

Since there are many resturants in the banglore , we will try to find the area where  there are less crowd. And We will also prefer that it wil be close to the city center

We Will use ML algorithm to generate most promising neighbourhood.


**Data Extarction**

The list of factors which will influence our decisions are:
*   Number of existing resturants in the neighbourhood.
*   Number of italina resturant 
*   Distance of the neighbor from the city



**Neighbor Candidates**
We will create a grid of cells covering area of interest which is approx 12x12 KM centerd arounf Banglore center.

In [1]:
!pip install geopy

Importing all the necessary libraries

In [0]:
import ssl
import sys
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="banglore_loc")
location = geolocator.geocode("Bangalore")

From the geopy library e are able to find the latitude and longitude of Banglore

In [3]:
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))
bang_lat,bang_long = location.latitude, location.longitude

Latitude = 12.9791198, Longitude = 77.5912997


As we have find the latitude and longitude of the area , equally spaced , centered around city  and within 6Km from Banglore. We will distribute the neighbour of 300 m so that the distance will be 600m between two neighbour.

In [4]:
!pip install shapely
!pip install pyproj

     |████████████████████████████████| 10.1MB 2.8MB/s 


We have to convert the data into 2D cartesian .We need the coordinate in X,Y cordinate because we are planning to make a hexagonal frid of the area,

In [5]:
import shapely.geometry
import pyproj
import math

#For converion in 2D
def longlot_to_xy(lon,lat):
  proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
  proj_xy = pyproj.Proj(proj='utm',zone=33,datum='WGS84')
  xy = pyproj.transform(proj_latlon,proj_xy,lon,lat)
  return xy[0],xy[1]


#Vice-versa
def xy_to_langlot(x,y):
  proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
  proj_xy = pyproj.Proj(proj='utm',zone=33,datum='WGS84')
  lonlat = pyproj.transform(proj_xy,proj_latlon,x,y)
  return lonlat[0],lonlat[1]

# To find the distance between two neighbours
def calc_xy_distance(x1,y1,x2,y2):
  dx = x2-x1
  dy = y2-y1
  return math.sqrt(dx*dx+dy*dy)

#For checking purpose
print('Banglore centre latitude={}, longitude={}'.format(bang_lat,bang_long))
x,y = longlot_to_xy(bang_lat,bang_long)
print('Banglore center UTM x={}, y={}'.format(x,y))
lon,lat = xy_to_langlot(x,y)
print('Banglore centre latitude={}, longitude={}'.format(lat,lon))

Banglore centre latitude=12.9791198, longitude=77.5912997
Banglore center UTM x=451532.50057396025, y=8613593.967979895
Banglore centre latitude=77.5912997, longitude=12.9791198


**Now lets create hexagoanl grid such that it is equally distant from each other**

In [6]:
#Here we will store the lat and longitude of the all the hexagonal grid
bang_cen_x,bang_cen_y = longlot_to_xy(bang_long,bang_lat)#In Cartesian form

k = math.sqrt(3)/2 # Vertical offset for horizonal grid
x_min = bang_cen_x - 6000 # since we are covering 6 Km
x_step = 600
y_min = bang_cen_y - 6000- (int(21/k)*k*600-12000)/2
y_step = 600*k

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(bang_cen_x, bang_cen_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_langlot(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [7]:
# Vizualization
!pip install folium

In [0]:
import folium

Now we will plot the map of Banglore. On top of that we will draw circle for individaul segments which are having radius of 300m.

In [9]:
map_bang = folium.Map(location = [bang_lat,bang_long],zoom_start =12)
folium.Marker([bang_lat,bang_long],popup='Banglore').add_to(map_bang)
for lat,lon in zip(latitudes,longitudes):#latitudes and longitudes of the hexagon defined in the above part
  folium.Circle([lat,lon],radius=300,color='blue',fill=False).add_to(map_bang)
map_bang

Checking the reverse order. i.e by putting the latitudes and longitudes , we are able to find exact adress

In [10]:
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
geolocator = Nominatim(user_agent="bang_address")
def get_address(latitudes,longitudes):
  location = geolocator.reverse([latitudes,longitudes])
  return location.address

addrr = get_address(12.958820695085988,77.57312796930427)  
print('The address is {}'.format(addrr))

The address is Fort High School, Krishna Rajendra Road, Mavalli, Vishveshwara Puram, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560 002, India


In [11]:
 pip install python-googlegeocoder

We are using only 60 records in one loop because of computation problem. For one loop if i am putting all the records it is giving me error.

In [13]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes[0:60], longitudes[0:60]):
    address = get_address(lat,lon)
    if address is None:
         address = 'NO ADDRESS'
    address = address.replace(', India', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
for lat, lon in zip(latitudes[61:120], longitudes[61:120]):
    address = get_address(lat,lon)
    if address is None:
         address = 'NO ADDRESS'
    address = address.replace(', India','') # We don't need country part of address
    addresses.append(address) 
    print(' .', end='')   
for lat, lon in zip(latitudes[121:160], longitudes[121:160]):
    address = get_address(lat,lon)
    if address is None:
         address = 'NO ADDRESS'
    address = address.replace(', India','') # We don't need country part of address
    addresses.append(address) 
    print(' .', end='')       
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


Getting the addresses in the neighbourhood.

In [14]:
addresses[50:150]

['ರಾಜಣ್ಣ ಲೇನ್, Ganigarpet, Dharmaraya Swamy Temple Ward, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560053',
 'Dasappa Maternity Hospital, Pehalwan Krishnappa Road, Ganigarpet, Dharmaraya Swamy Temple Ward, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560002',
 "Bishop Cotton's Women's Christian College, 19, 3rd Cross Road, Sudhama Nagara, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560027",
 'Government Pharmacy College, Kalinga Rao Road, Sudhama Nagara, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560027',
 'Sudhama Nagara, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560027',
 'Utsav, Shanti Road, Shanthi Nagar, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560027"',
 'Richmond Town, Shanthi Nagar, East Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560027"',
 'Richmond Town, Shanthi Nagar, East Zone, Bengaluru, Bangalore North, Ba

In [15]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses[0:155],
                             'Latitude': latitudes[0:155],
                             'Longitude': longitudes[0:155],
                             'X': xs[0:155],
                             'Y': ys[0:155],
                             'Distance from center': distances_from_center[0:155]})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Fort High School, Krishna Rajendra Road, Maval...",12.958821,77.573128,8.885556e+06,2.961840e+06,5992.495307
1,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.957730,77.575652,8.886156e+06,2.961840e+06,5840.376700
2,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.956640,77.578175,8.886756e+06,2.961840e+06,5747.173218
3,"Hindustan Petroleum, Proposed Hudson Cricle - ...",12.955549,77.580699,8.887356e+06,2.961840e+06,5715.767665
4,"New car designer, Lalbagh Fort Road, Mavalli, ...",12.954459,77.583222,8.887956e+06,2.961840e+06,5747.173218
5,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.953369,77.585744,8.888556e+06,2.961840e+06,5840.376700
6,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.952279,77.588267,8.889156e+06,2.961840e+06,5992.495307
7,"St. Joseph's School, Cottonpete Main Road, K R...",12.962601,77.570305,8.884656e+06,2.962360e+06,5855.766389
8,"Bangalore Medical College Hospitals, ಸುಲ್ತಾನ್ ...",12.961510,77.572829,8.885256e+06,2.962360e+06,5604.462508
9,"Srinidhi Sagar, 2, Krishna Rajendra Road, K R ...",12.960419,77.575353,8.885856e+06,2.962360e+06,5408.326913


In [0]:
df_locations.to_pickle('./locations.pkl')

Now with the help of foursquare api, we will find the list of resturants and then list of italuan resturant in the area.

In [0]:
client_id = 'WQKMV45IUJFHR204GLPOFWAO4AWDFRTMISHNLDRHOLKD2FO4'
client_secret = 'UWBSPVL1GOZO1OLP3JYWNOBH2CDQ545L13MK5LJBGANSZ4SF'
version = '20180604'

In [0]:
import requests 
limit =100
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Fetching all the venues

In [20]:
bangalore_venues = getNearbyVenues(names=df_locations['Address'],
                                   latitudes=df_locations['Latitude'],
                                   longitudes=df_locations['Longitude'],
                                  )

Fort High School, Krishna Rajendra Road, Mavalli, Vishveshwara Puram, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560 002
Dharmaraya Swamy Temple Ward, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560 002
Dharmaraya Swamy Temple Ward, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560 002
Hindustan Petroleum, Proposed Hudson Cricle - Minerva Circle Flyover, Sudhama Nagara, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560 002
New car designer, Lalbagh Fort Road, Mavalli, Vishveshwara Puram, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka, 560 002
Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mavalli, Vishveshwara Puram, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka
Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mavalli, Vishveshwara Puram, South Zone, Bengaluru, Bangalore North, Bangalore Urban, Karnataka
St. Joseph's School, Cottonpete Main Road, K R Market Ward, We

From foursq

In [0]:
food_category = '4d4b7105d754a06374d81259'

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

In [0]:
def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'hotel', 'dining', 'bites']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

# New Section

In [23]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = longlot_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_351.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_351.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_351.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [24]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 452
Total number of Italian restaurants: 14
Percentage of Italian restaurants: 3.10%
Average number of restaurants in neighborhood: 3.456043956043956


In [25]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4f093e86e4b0ef60ee6d3df4', 'Prakash cafe', 12.96049019646807, 77.57102892400674, 'India', 225, False, 8884988.517896798, 2961995.196752784)
('524d2f9f8bbd97b6e12fecd9', 'Classic Caterers', 12.957893430072096, 77.57694661617279, 'No.13/11 A M Road,Kalasipalyam, Bangalore 560002, Karnātaka, India', 330, False, 8886398.906232743, 2961987.157351209)
('4f7a853ae4b0d859b60c9715', 'BIT Canteen', 12.95481286570614, 77.57455121517431, 'India', 346, False, 8886197.464145478, 2961151.647999301)
('4d85719d7e8ef04d03be21be', 'NMH Tiffin House', 12.954300016820962, 77.5788060159709, 'Near Minerva Circle, VV Puram (Rama Iyengar Rd), Bangalore, Karnātaka, India', 247, False, 8887090.566945955, 2961420.7296396075)
('4d29ccc66e27a14317362824', 'Hotel Nandhini', 12.955307313850327, 77.5794984145012, 'No. 114/2, Near Minerva Circle (L.F. Road), Bangalore, Karnātaka, India', 340, False, 8887138.346942313, 2961685.989060371)
('4d01ec0e3be26dcb537475e4', 'May

In [26]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('534ffaca498e9ea777d34e9b', 'Ottimo', 12.967041213267786, 77.59563043238367, 'ITC Gardenia, Bangalore, Karnātaka, India', 342, True, 8889300.258001408, 2965484.8211480663)
('4bc1cd90b492d13a4e74a660', 'Toscano', 12.97198038085137, 77.59606565238312, 'UB City Level 2 Concorde Block (24 Vittal Mallya Road), Bangalore, Karnātaka, India', 276, True, 8888944.322836278, 2966525.8619835787)
('524c709211d27c7b392a7897', 'Alba', 12.97254127003947, 77.59459436597112, 'JW Marriott Bengaluru (Vittal Mallya Road), Bangalore, Karnātaka, India', 264, True, 8888601.126964148, 2966510.4179724376)
('5050aec7e4b0dd1afe0a1f4e', 'The Tower Kitchen', 12.971671379872115, 77.5963634706781, 'India', 280, True, 8889031.31274633, 2966489.3053250257)
('5110cd35e4b0162f3fc169ee', 'Cafe Mangii', 12.971984856069145, 77.59597131547177, 'UB CITY, India', 283, True, 8888925.139084797, 2966518.478977431)
('526b900d11d267e76ab188f4', 'Café Mangii', 12.9719497259831

In [27]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Hotel Fishland, Sukh Sagar GandhiNagar, Chicken County, Kamat Yatrinivas, Sukh Sagar Food Court, navayuga, Kamat Hotel, Talk Of The Town
Restaurants around location 102: Hotel Fishland, Vasudev Adigas, Adigas Hotel, Sukh Sagar GandhiNagar, Kamat Yatrinivas, New Shanti Sagar, Sukh Sagar Food Court
Restaurants around location 103: Vasudev Adigas, Adigas Hotel
Restaurants around location 104: Kaveri Bhavan, Cubbonpet Biriyani Bazaar
Restaurants around location 105: Bhagini, Rajdhani, Kaveri Bhavan
Restaurants around location 106: Rajdhani
Restaurants around location 107: Palate Corner, Konark Kanteerava
Restaurants around location 108: Kebabs & Kurries, Kudla, EDO, Palate Corner, Ottimo, Legacy Of Punjab, Woody's
Restaurants around location 109: Kebabs & Kurries, Mahesh Lunch Home, Kudla, EDO, Kazé, Sunheri Restaurant, Aromas of China, Woody's
Restaurants around location 110: Harima, Mahesh Lunch Home

In [28]:
map_bang = folium.Map(location = [bang_lat,bang_long],zoom_start =12)
folium.Marker([bang_lat,bang_long],popup='Banglore').add_to(map_bang)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_bang)
map_bang

In [29]:
location_restaurants_count = [len(res) for res in location_restaurants]
# location_restaurants_count
df_locations['Restaurants in area'] = location_restaurants_count[0:155]

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 3.456043956043956


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"Fort High School, Krishna Rajendra Road, Maval...",12.958821,77.573128,8.885556e+06,2.961840e+06,5992.495307,1
1,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.957730,77.575652,8.886156e+06,2.961840e+06,5840.376700,1
2,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.956640,77.578175,8.886756e+06,2.961840e+06,5747.173218,4
3,"Hindustan Petroleum, Proposed Hudson Cricle - ...",12.955549,77.580699,8.887356e+06,2.961840e+06,5715.767665,4
4,"New car designer, Lalbagh Fort Road, Mavalli, ...",12.954459,77.583222,8.887956e+06,2.961840e+06,5747.173218,1
5,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.953369,77.585744,8.888556e+06,2.961840e+06,5840.376700,3
6,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.952279,77.588267,8.889156e+06,2.961840e+06,5992.495307,0
7,"St. Joseph's School, Cottonpete Main Road, K R...",12.962601,77.570305,8.884656e+06,2.962360e+06,5855.766389,1
8,"Bangalore Medical College Hospitals, ಸುಲ್ತಾನ್ ...",12.961510,77.572829,8.885256e+06,2.962360e+06,5604.462508,1
9,"Srinidhi Sagar, 2, Krishna Rajendra Road, K R ...",12.960419,77.575353,8.885856e+06,2.962360e+06,5408.326913,1


In [0]:
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant[0:155]

In [31]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Italian restaurant
0,"Fort High School, Krishna Rajendra Road, Maval...",12.958821,77.573128,8.885556e+06,2.961840e+06,5992.495307,1,5225.267649
1,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.957730,77.575652,8.886156e+06,2.961840e+06,5840.376700,1,4813.516635
2,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.956640,77.578175,8.886756e+06,2.961840e+06,5747.173218,4,4444.824269
3,"Hindustan Petroleum, Proposed Hudson Cricle - ...",12.955549,77.580699,8.887356e+06,2.961840e+06,5715.767665,4,4130.736395
4,"New car designer, Lalbagh Fort Road, Mavalli, ...",12.954459,77.583222,8.887956e+06,2.961840e+06,5747.173218,1,3884.521019
5,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.953369,77.585744,8.888556e+06,2.961840e+06,5840.376700,3,3719.680623
6,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.952279,77.588267,8.889156e+06,2.961840e+06,5992.495307,0,3647.265321
7,"St. Joseph's School, Cottonpete Main Road, K R...",12.962601,77.570305,8.884656e+06,2.962360e+06,5855.766389,1,5597.880918
8,"Bangalore Medical College Hospitals, ಸುಲ್ತಾನ್ ...",12.961510,77.572829,8.885256e+06,2.962360e+06,5604.462508,1,5111.045994
9,"Srinidhi Sagar, 2, Krishna Rajendra Road, K R ...",12.960419,77.575353,8.885856e+06,2.962360e+06,5408.326913,1,4650.732366


In [0]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]

In [0]:
good_latitudes = df_locations['Latitude'].values
good_longitudes = df_locations['Longitude'].values

In [35]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 2620.790324034266


In [37]:
from folium.plugins import HeatMap
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_langlot(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_bang = folium.Map(location= [bang_lat,bang_long], zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_bang)
HeatMap(restaurant_latlons).add_to(map_bang)
folium.Circle( [bang_lat,bang_long], radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_bang)
folium.Marker( [bang_lat,bang_long]).add_to(map_bang)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_bang) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_bang)
#folium.GeoJson(berlin_boroughs, style_function=boroughs_style, name='geojson').add_to(map_berlin)
map_bang

In [38]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Italian restaurant
0,"Fort High School, Krishna Rajendra Road, Maval...",12.958821,77.573128,8.885556e+06,2.961840e+06,5992.495307,1,5225.267649
1,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.957730,77.575652,8.886156e+06,2.961840e+06,5840.376700,1,4813.516635
2,"Dharmaraya Swamy Temple Ward, South Zone, Beng...",12.956640,77.578175,8.886756e+06,2.961840e+06,5747.173218,4,4444.824269
3,"Hindustan Petroleum, Proposed Hudson Cricle - ...",12.955549,77.580699,8.887356e+06,2.961840e+06,5715.767665,4,4130.736395
4,"New car designer, Lalbagh Fort Road, Mavalli, ...",12.954459,77.583222,8.887956e+06,2.961840e+06,5747.173218,1,3884.521019
5,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.953369,77.585744,8.888556e+06,2.961840e+06,5840.376700,3,3719.680623
6,"Lalbagh Botanical Gardens, ಪಾದಚಾರಿ ಮಾರ್ಗ, Mava...",12.952279,77.588267,8.889156e+06,2.961840e+06,5992.495307,0,3647.265321
7,"St. Joseph's School, Cottonpete Main Road, K R...",12.962601,77.570305,8.884656e+06,2.962360e+06,5855.766389,1,5597.880918
8,"Bangalore Medical College Hospitals, ಸುಲ್ತಾನ್ ...",12.961510,77.572829,8.885256e+06,2.962360e+06,5604.462508,1,5111.045994
9,"Srinidhi Sagar, 2, Krishna Rajendra Road, K R ...",12.960419,77.575353,8.885856e+06,2.962360e+06,5408.326913,1,4650.732366


**Results and Discussion**

Our analysis shows that although there is a great number of restaurants in Banglore (~2000 in our initial area of interest which was 12x12km around Banglore), there are pockets of low restaurant density fairly close to city center. Highest concentration of restaurants was detected north and west from Banglore, so we focused our attention to areas south, south-east and east, corresponding to and south-east corner of central area. 


Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this  zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Banglore center but not crowded with existing restaurants (particularly Italian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

**Conclusion**

Purpose of this project was to identify Banglore areas close to center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis , and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.